# Building an Intelligent Search Agent with LangChain

## 1. Setting Up the Environment

First, let's import the necessary packages and set up our environment variables.

In [1]:
%pip install python-dotenv langchain langchain-community langchain_groq

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
from dotenv import load_dotenv
from langchain.agents import Tool, AgentExecutor, create_react_agent
from langchain.tools import DuckDuckGoSearchRun
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory

# Load environment variables
load_dotenv()

# Initialize OpenAI API key
os.getenv("Groq_API_KEY")

## 2. Creating the Search Tool

We'll create a search tool using DuckDuckGo's search API. This will allow our agent to search the internet for information.

In [3]:
# Initialize the search tool
search = DuckDuckGoSearchRun()

tools = [
    Tool(
        name="Search",
        func=search.run,
        description="Useful for searching information on the internet. Use this when you need to find current or factual information."
    )
]

## 3. Setting Up the Language Model

We'll use OpenAI's GPT model as our agent's brain. This will help process and understand both the user's queries and the search results.

In [4]:
# Initialize the language model
llm = ChatGroq(
    temperature=0.7,
    model="llama3-8b-8192",
    api_key=os.getenv("Groq_API_KEY"),
)

## 4. Creating the Agent

Now we'll create our agent by combining the search tool with the language model.

In [5]:
# Define the prompt template for the agent
template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought: {agent_scratchpad}"""

# Create the agent
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=PromptTemplate.from_template(template)
)

# Create the agent executor
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
    memory=ConversationBufferMemory(memory_key="chat_history")
)

C:\Users\Dark_Coder\AppData\Local\Temp\ipykernel_13532\167193580.py:34: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory=ConversationBufferMemory(memory_key="chat_history")


## 5. Testing the Search Agent

Let's test our agent with some example queries to see how it performs.

In [6]:
def ask_question(query):
    """
    Function to ask a question to the agent and get a response
    """
    try:
        response = agent_executor.invoke({"input": query})
        return response.get("output", "Sorry, I couldn't generate a response.")
    except Exception as e:
        return f"An error occurred: {str(e)}"


In [7]:
# Test the agent with a query
query = "What are the latest developments in quantum computing?"
print(ask_question(query))



> Entering new AgentExecutor chain...
Question: What are the latest developments in quantum computing?

Thought: I should use the Search tool to find the latest information on quantum computing developments.

Action: Search

Action Input: "latest developments in quantum computing"
MIT physicists predict exotic form of matter with potential for quantum computing. New work suggests the ability to create fractionalized electrons known as non-Abelian anyons without a magnetic field, opening new possibilities for basic research and future applications. November 18, 2024. Read full story → Read the latest about the development of quantum computers. ... 2024 — Predicting the behavior of many interacting quantum particles is a complicated process but is key to harness quantum ... Quantum computing is in its infancy, but its potential has sparked intense exploration. This article examines three major trajectories that will shape the future of quantum computing: the gate ... Breakthrough in Ma

In [8]:
%pip install graphviz Pillow

Note: you may need to restart the kernel to use updated packages.


In [10]:
import os
from PIL import Image, ImageDraw, ImageFont
from graphviz import Digraph

def create_step_diagram(step):
    dot = Digraph(comment=f'Step {step}', format='png')
    dot.attr(rankdir='LR', size='12,8')
    dot.attr('node', shape='box', style='rounded,filled', fontname='Arial', fontsize='12')
    dot.attr('edge', fontname='Arial', fontsize='10')

    components = {
        'user': 'User Query',
        'agent': 'ReAct Agent',
        'executor': 'Agent Executor',
        'llm': 'Groq LLM',
        'tools': 'DuckDuckGo Search',
        'memory': 'ConversationBuffer\nMemory',
        'prompt': 'Prompt Template'
    }

    highlights = {
        1: ['user', 'executor'],
        2: ['executor', 'agent', 'prompt'],
        3: ['agent', 'llm'],
        4: ['agent', 'tools'],
        5: ['executor', 'memory'],
        6: ['memory', 'agent']
    }

    for node_id, label in components.items():
        color = 'yellow' if node_id in highlights.get(step, []) else 'lightblue'
        dot.node(node_id, label, fillcolor=color)

    dot.edge('user', 'executor', 'Query')
    dot.edge('executor', 'agent', 'Execute')
    dot.edge('agent', 'llm', 'Process')
    dot.edge('agent', 'tools', 'Use')
    dot.edge('prompt', 'agent', 'Format')
    dot.edge('executor', 'memory', 'Store/Retrieve')
    dot.edge('memory', 'agent', 'Context')

    return dot

def add_step_label(image, step):
    draw = ImageDraw.Draw(image)
    font = ImageFont.truetype("arial.ttf", 36)
    draw.text((10, 10), f"Step {step}", font=font, fill=(0, 0, 0))
    return image

def create_animation():
    frames = []
    for step in range(1, 7):
        dot = create_step_diagram(step)
        dot.render(f'step_{step}', cleanup=True)
        img = Image.open(f'step_{step}.png')
        img = add_step_label(img, step)
        frames.append(img)

    frames[0].save(
        'langchain_flow_improved.gif',
        save_all=True,
        append_images=frames[1:],
        duration=1500,
        loop=0
    )

    for step in range(1, 7):
        os.remove(f'step_{step}.png')

if __name__ == "__main__":
    create_animation()